In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['LANGSMITH_TRACING'] = 'true'

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = 'o1-mini')
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000012808620050>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000128083FC1D0>, root_client=<openai.OpenAI object at 0x0000012805662990>, root_async_client=<openai.AsyncOpenAI object at 0x00000128084A3990>, model_name='o1-mini', temperature=1.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [4]:
from langchain_community.document_loaders import WikipediaLoader

loader = WikipediaLoader(
  query='Talha Anjum',
  load_max_docs=2,
  lang='en'
)

In [7]:
docs = loader.load()

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=450)

text_docs = splitter.split_documents(docs)

In [11]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

vector_store = FAISS.from_documents(text_docs, embeddings)

In [12]:
retriver = vector_store.as_retriever()

In [17]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
  input_variables=['question', 'context'],

  template='''Answer the following based on the given context in english language:
  <context>
  {context}
  </context>

  Question: {question}
  Answer:
  '''


)

In [18]:
from langchain.chains.combine_documents import create_stuff_documents_chain

chain = create_stuff_documents_chain(
  llm=llm,
  prompt=prompt,
)

In [ ]:
question = 'Kaun Talha '
context = retriver.get_relevant_documents(question, top_k=1)
response = chain.invoke(
  {
    'question': question,
    'context': context
  }
)

print(response)

Talha Anjum, born on October 3, 1995, in Karachi, Sindh, Pakistan, is a prominent Pakistani rapper, singer, songwriter, and lyricist. He is best known as a co-founder and member of the hip-hop duo **Young Stunners**, alongside Talhah Yunus. Talha gained widespread recognition with the release of their breakout song "**Burger-e-Karachi**" in 2013. Some of his major hits include "**Tum Tum**", "**Gumaan**", "**Don't Mind**", "**Afsanay**", "**Downers at Dusk**", and "**Groove Mera**", which served as the anthem for PSL 6.

Educated at the Army Public School in Karachi, Talha began his career by participating in rap battles during his school years, using his lyrics to address fundamental social issues and connect with audiences. Alongside Talhah Yunus, he played a key role in introducing Urdu rap to the mainstream Pakistani music scene. Their music is primarily produced by Jokhay, and they have collaborated with various artists both within Pakistan and internationally.

In addition to his